In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")


In [2]:
data_path = "../data/raw/movie_metadata.csv"
df = pd.read_csv(data_path, encoding='latin1')
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


***Building final dataset***

In [3]:
df['is_good'] = (df['imdb_score'] >= 7)


In [4]:
numeric_features = [
    "budget",
    "gross",
    "duration",
    "num_critic_for_reviews",
    "num_user_for_reviews",
    "num_voted_users",
    "title_year",
]
categorical_features = [
    "color",
    "genres",
    "language",
    "country",
    "content_rating",
]
y  = ["is_good"]

selected_columns = numeric_features + categorical_features + y

In [5]:
df_subset = df[selected_columns].copy()

In [6]:
df_subset

,budget,gross,duration,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,color,genres,language,country,content_rating,is_good
0,237000000.0,760505847.0,178.0,723.0,3054.0,886204,2009.0,Color,Action|Adventure|Fantasy|Sci-Fi,English,USA,PG-13,True
1,300000000.0,309404152.0,169.0,302.0,1238.0,471220,2007.0,Color,Action|Adventure|Fantasy,English,USA,PG-13,True
2,245000000.0,200074175.0,148.0,602.0,994.0,275868,2015.0,Color,Action|Adventure|Thriller,English,UK,PG-13,False
3,250000000.0,448130642.0,164.0,813.0,2701.0,1144337,2012.0,Color,Action|Thriller,English,USA,PG-13,True
4,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,Documentary,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,NaN,NaN,87.0,1.0,6.0,629,2013.0,Color,Comedy|Drama,English,Canada,NaN,True
5039,NaN,NaN,43.0,43.0,359.0,73839,NaN,Color,Crime|Drama|Mystery|Thriller,English,USA,TV-14,True
5040,1400.0,NaN,76.0,13.0,3.0,38,2013.0,Color,Drama|Horror|Thriller,English,USA,NaN,False
5041,NaN,10443.0,100.0,14.0,9.0,1255,2012.0,Color,Comedy|Drama|Romance,English,USA,PG-13,False


In [7]:
df_subset.isna().sum()

budget                    492
gross                     884
duration                   15
num_critic_for_reviews     50
num_user_for_reviews       21
num_voted_users             0
title_year                108
color                      19
genres                      0
language                   14
country                     5
content_rating            303
is_good                     0
dtype: int64

In [8]:
df_subset.color.unique()

array(['Color', nan, ' Black and White'], dtype=object)

In [9]:
# Filling NaNs
for col in numeric_features:
    df_subset[col] = df_subset[col].fillna(df_subset[col].median())

# Fill categorical NaNs with mode
for col in categorical_features:
    df_subset[col] = df_subset[col].fillna(df_subset[col].mode()[0])


In [10]:
df_subset.isna().sum()


budget                    0
gross                     0
duration                  0
num_critic_for_reviews    0
num_user_for_reviews      0
num_voted_users           0
title_year                0
color                     0
genres                    0
language                  0
country                   0
content_rating            0
is_good                   0
dtype: int64

In [11]:
df_subset.color.unique()

array(['Color', ' Black and White'], dtype=object)

In [12]:
df_subset["color"] = df_subset["color"].apply(lambda x: True if x== "Color" else False)

In [13]:
df_subset.country.unique()

array(['USA', 'UK', 'New Zealand', 'Canada', 'Australia', 'Belgium',
       'Japan', 'Germany', 'China', 'France', 'New Line', 'Mexico',
       'Spain', 'Hong Kong', 'Czech Republic', 'India', 'Soviet Union',
       'South Korea', 'Peru', 'Italy', 'Russia', 'Aruba', 'Denmark',
       'Libya', 'Ireland', 'South Africa', 'Iceland', 'Switzerland',
       'Romania', 'West Germany', 'Chile', 'Netherlands', 'Hungary',
       'Panama', 'Greece', 'Sweden', 'Norway', 'Taiwan', 'Official site',
       'Cambodia', 'Thailand', 'Slovakia', 'Bulgaria', 'Iran', 'Poland',
       'Georgia', 'Turkey', 'Nigeria', 'Brazil', 'Finland', 'Bahamas',
       'Argentina', 'Colombia', 'Israel', 'Egypt', 'Kyrgyzstan',
       'Indonesia', 'Pakistan', 'Slovenia', 'Afghanistan',
       'Dominican Republic', 'Cameroon', 'United Arab Emirates', 'Kenya',
       'Philippines'], dtype=object)

In [14]:
country_counts = df_subset['country'].value_counts().sort_values(ascending=False)
print(country_counts)

country
USA                     3812
UK                       448
France                   154
Canada                   126
Germany                   97
                        ... 
Dominican Republic         1
Cameroon                   1
United Arab Emirates       1
Kenya                      1
Philippines                1
Name: count, Length: 65, dtype: int64


In [15]:
country_fix_map = {
    "West Germany": "Germany",
    "Soviet Union": "Russia",
    "Hong Kong": "China",
    "Official site": "Other",  # remove invalid entries
    "New Line": "Other",       # remove invalid entries
}
df_subset['country'] = df_subset['country'].replace(country_fix_map)

In [16]:
country_counts = df_subset['country'].value_counts().sort_values(ascending=False)
print(country_counts)

country
USA                     3812
UK                       448
France                   154
Canada                   126
Germany                  100
                        ... 
Dominican Republic         1
Cameroon                   1
United Arab Emirates       1
Kenya                      1
Philippines                1
Name: count, Length: 61, dtype: int64


In [17]:
# Identify countries with fewer than 20 entries and replace them with "Other"
rare_countries = country_counts[country_counts < 20].index
df_subset['country'] = df_subset['country'].replace(rare_countries, "Other")

In [18]:
country_counts = df_subset['country'].value_counts().sort_values(ascending=False)
print(country_counts)

country
USA          3812
UK            448
Other         188
France        154
Canada        126
Germany       100
Australia      55
China          47
India          34
Spain          33
Japan          23
Italy          23
Name: count, dtype: int64


In [19]:
language_counts = df_subset['language'].value_counts().sort_values(ascending=False)
print(language_counts)

language
English       4718
French          73
Spanish         40
Hindi           28
Mandarin        26
German          19
Japanese        18
Cantonese       11
Italian         11
Russian         11
Korean           8
Portuguese       8
Danish           5
Swedish          5
Hebrew           5
Arabic           5
Polish           4
Persian          4
Norwegian        4
Dutch            4
Chinese          3
Thai             3
Indonesian       2
Romanian         2
Zulu             2
Dari             2
Icelandic        2
Aboriginal       2
Filipino         1
Maya             1
Kazakh           1
Telugu           1
Aramaic          1
Mongolian        1
Bosnian          1
Hungarian        1
Kannada          1
Czech            1
Dzongkha         1
Tamil            1
Panjabi          1
Vietnamese       1
Urdu             1
Slovenian        1
Greek            1
Swahili          1
Name: count, dtype: int64


In [20]:
# Identify languages with fewer than 20 entries and replace them with "Other"
rare_languages = language_counts[language_counts < 20].index
# Replace them with "Other"
df_subset['language'] = df_subset['language'].replace(rare_languages, "Other")

In [21]:
language_counts = df_subset['language'].value_counts().sort_values(ascending=False)
print(language_counts)

language
English     4718
Other        158
French        73
Spanish       40
Hindi         28
Mandarin      26
Name: count, dtype: int64


In [22]:
content_rating_counts = df_subset['content_rating'].value_counts().sort_values(ascending=False)
print(content_rating_counts)

content_rating
R            2421
PG-13        1461
PG            701
Not Rated     116
G             112
Unrated        62
Approved       55
TV-14          30
TV-MA          20
TV-PG          13
X              13
TV-G           10
Passed          9
NC-17           7
GP              6
M               5
TV-Y            1
TV-Y7           1
Name: count, dtype: int64


In [23]:
rating_to_group = {
    # Kids
    "G": "Kids",
    "TV-Y": "Kids",
    "TV-Y7": "Kids",
    "TV-G": "Kids",
    "Approved": "Kids",
    "Passed": "Kids",

    # Young 
    "PG": "Young",
    "TV-PG": "Young",
    "GP": "Young",
    "M": "Young",

    # Teen
    "PG-13": "Teen",
    "TV-14": "Teen",

    # Adult
    "R": "Adult",
    "TV-MA": "Adult",

    # Explicit
    "NC-17": "Explicit",
    "X": "Explicit",

    # Other
    "Not Rated": "Other",
    "Unrated": "Other"
}


df_subset['content_rating'] = df_subset['content_rating'].replace(rating_to_group)


In [24]:
print(df_subset['content_rating'].value_counts())


content_rating
Adult       2441
Teen        1491
Young        725
Kids         188
Other        178
Explicit      20
Name: count, dtype: int64


In [25]:
df_subset

,budget,gross,duration,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,color,genres,language,country,content_rating,is_good
0,237000000.0,760505847.0,178.0,723.0,3054.0,886204,2009.0,True,Action|Adventure|Fantasy|Sci-Fi,English,USA,Teen,True
1,300000000.0,309404152.0,169.0,302.0,1238.0,471220,2007.0,True,Action|Adventure|Fantasy,English,USA,Teen,True
2,245000000.0,200074175.0,148.0,602.0,994.0,275868,2015.0,True,Action|Adventure|Thriller,English,UK,Teen,False
3,250000000.0,448130642.0,164.0,813.0,2701.0,1144337,2012.0,True,Action|Thriller,English,USA,Teen,True
4,20000000.0,25517500.0,103.0,110.0,156.0,8,2005.0,True,Documentary,English,USA,Adult,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,20000000.0,25517500.0,87.0,1.0,6.0,629,2013.0,True,Comedy|Drama,English,Canada,Adult,True
5039,20000000.0,25517500.0,43.0,43.0,359.0,73839,2005.0,True,Crime|Drama|Mystery|Thriller,English,USA,Teen,True
5040,1400.0,25517500.0,76.0,13.0,3.0,38,2013.0,True,Drama|Horror|Thriller,English,USA,Adult,False
5041,20000000.0,10443.0,100.0,14.0,9.0,1255,2012.0,True,Comedy|Drama|Romance,English,USA,Teen,False


In [26]:
# Multi-hot encoding for genres
df_subset["genres"] = df_subset["genres"].fillna("").apply(lambda x: x.split("|"))
all_genres = sorted({g for sublist in df_subset["genres"] for g in sublist if g != ""})
for genre in all_genres:
    df_subset[f"genre_{genre}"] = df_subset["genres"].apply(lambda lst: genre in lst)

df_subset = df_subset.drop(columns=["genres"])


In [28]:
all_genres

['Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Thriller',
 'War',
 'Western']

In [29]:
df_subset

,budget,gross,duration,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,color,language,country,...,genre_Mystery,genre_News,genre_Reality-TV,genre_Romance,genre_Sci-Fi,genre_Short,genre_Sport,genre_Thriller,genre_War,genre_Western
0,237000000.0,760505847.0,178.0,723.0,3054.0,886204,2009.0,True,English,USA,...,False,False,False,False,True,False,False,False,False,False
1,300000000.0,309404152.0,169.0,302.0,1238.0,471220,2007.0,True,English,USA,...,False,False,False,False,False,False,False,False,False,False
2,245000000.0,200074175.0,148.0,602.0,994.0,275868,2015.0,True,English,UK,...,False,False,False,False,False,False,False,True,False,False
3,250000000.0,448130642.0,164.0,813.0,2701.0,1144337,2012.0,True,English,USA,...,False,False,False,False,False,False,False,True,False,False
4,20000000.0,25517500.0,103.0,110.0,156.0,8,2005.0,True,English,USA,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,20000000.0,25517500.0,87.0,1.0,6.0,629,2013.0,True,English,Canada,...,False,False,False,False,False,False,False,False,False,False
5039,20000000.0,25517500.0,43.0,43.0,359.0,73839,2005.0,True,English,USA,...,True,False,False,False,False,False,False,True,False,False
5040,1400.0,25517500.0,76.0,13.0,3.0,38,2013.0,True,English,USA,...,False,False,False,False,False,False,False,True,False,False
5041,20000000.0,10443.0,100.0,14.0,9.0,1255,2012.0,True,English,USA,...,False,False,False,True,False,False,False,False,False,False


In [30]:
categorical_simple = ["language", "country", "content_rating"]

df_subset = pd.get_dummies(
    df_subset,
    columns=categorical_simple,
    dummy_na=False
)


In [31]:
df_subset.columns.value_counts()

budget                     1
gross                      1
duration                   1
num_critic_for_reviews     1
num_user_for_reviews       1
num_voted_users            1
title_year                 1
color                      1
is_good                    1
genre_Action               1
genre_Adventure            1
genre_Animation            1
genre_Biography            1
genre_Comedy               1
genre_Crime                1
genre_Documentary          1
genre_Drama                1
genre_Family               1
genre_Fantasy              1
genre_Film-Noir            1
genre_Game-Show            1
genre_History              1
genre_Horror               1
genre_Music                1
genre_Musical              1
genre_Mystery              1
genre_News                 1
genre_Reality-TV           1
genre_Romance              1
genre_Sci-Fi               1
genre_Short                1
genre_Sport                1
genre_Thriller             1
genre_War                  1
genre_Western 

In [32]:
df_subset.to_csv("../data/processed/movie_metadata_processed.csv", index=False)

In [33]:
df_subset

,budget,gross,duration,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,color,is_good,genre_Action,...,country_Other,country_Spain,country_UK,country_USA,content_rating_Adult,content_rating_Explicit,content_rating_Kids,content_rating_Other,content_rating_Teen,content_rating_Young
0,237000000.0,760505847.0,178.0,723.0,3054.0,886204,2009.0,True,True,True,...,False,False,False,True,False,False,False,False,True,False
1,300000000.0,309404152.0,169.0,302.0,1238.0,471220,2007.0,True,True,True,...,False,False,False,True,False,False,False,False,True,False
2,245000000.0,200074175.0,148.0,602.0,994.0,275868,2015.0,True,False,True,...,False,False,True,False,False,False,False,False,True,False
3,250000000.0,448130642.0,164.0,813.0,2701.0,1144337,2012.0,True,True,True,...,False,False,False,True,False,False,False,False,True,False
4,20000000.0,25517500.0,103.0,110.0,156.0,8,2005.0,True,True,False,...,False,False,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,20000000.0,25517500.0,87.0,1.0,6.0,629,2013.0,True,True,False,...,False,False,False,False,True,False,False,False,False,False
5039,20000000.0,25517500.0,43.0,43.0,359.0,73839,2005.0,True,True,False,...,False,False,False,True,False,False,False,False,True,False
5040,1400.0,25517500.0,76.0,13.0,3.0,38,2013.0,True,False,False,...,False,False,False,True,True,False,False,False,False,False
5041,20000000.0,10443.0,100.0,14.0,9.0,1255,2012.0,True,False,False,...,False,False,False,True,False,False,False,False,True,False


df_subset

In [34]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   budget                   5043 non-null   float64
 1   gross                    5043 non-null   float64
 2   duration                 5043 non-null   float64
 3   num_critic_for_reviews   5043 non-null   float64
 4   num_user_for_reviews     5043 non-null   float64
 5   num_voted_users          5043 non-null   int64  
 6   title_year               5043 non-null   float64
 7   color                    5043 non-null   bool   
 8   is_good                  5043 non-null   bool   
 9   genre_Action             5043 non-null   bool   
 10  genre_Adventure          5043 non-null   bool   
 11  genre_Animation          5043 non-null   bool   
 12  genre_Biography          5043 non-null   bool   
 13  genre_Comedy             5043 non-null   bool   
 14  genre_Crime             